In [2]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime

In [158]:
# 파일명 수정 결과 즉석 확인
files = ["20450092_전순희_접수증_2022본553.pdf", "20450092_전순희_접수증_2022본553에혀.pdf", "20450092 전순희 접수증 2022본553 에혀.pdf", "20438985_정병수_기타행자부.pdf"]

# 기본 변수
docu_folder_dict = {"원인서류": "1.원인서류", "양도통지서": "2.양도통지서", "집행권원": "3.집행권원", "강제집행": "4.강제집행", "등본": "5.등초본",
              "초본": "5.등초본", "등초본":"5.등초본", "외국인증명": "5.등초본", "개인회생": "6.개인회생", "신용회복": "7.신용회복", "파산": "8.파산", "재산조사": "9.재산조사", "기타": "기타"}

# 필수 요소에 대한 컴파일
p_key = re.compile(r"([\d]{8})[\D]")  # match
docu_kind = r'원인서류|양도통지서[\s]?\d차|양도통지서|집행권원[\s]?[재]?[도]?|강제집행|등초본|(?<!등기부)등본|초본|외국인증명|개인회생|신용회복|파산|재산조사|기타'
p_docu = re.compile(docu_kind)
p_event = re.compile(r"\d{4}[ㄱ-ㅎ가-힣]{1,3}\d+")

p_basic = re.compile(r'(\d{8})[_\s]?(\D+.+)[_\s]?(' + docu_kind + r')') #event의 스타트 번호 >= basic의 엔드번호
p_out = re.compile('개인회생\(면책\)|파산\(면책\)|환매|매각|종결')

# 파일명 수정을 위한 컴파일--------------------------------
# 날짜
# 년월일 사이에 기호나 공백이 들어가 있는 경우로 찾아서 6자리로 맞춰준다.
# 괄호와 공백을 포함시켜서 모두 제거 대상으로하고 대체 단어에 앞공백만 추가
# 기호를 기준으로 쓰기 때문에 반드시 기호제거보다 나와야
p_day4 = re.compile(r'\s?\(?\s?(20|19)?(?P<y>\d{2})[.-/\s](?P<m>\d{1})[.-/\s](?P<d>\d{1})(?!\d)\s?\)?')
p_day5 = re.compile(r'\s?\(?\s?(20|19)?(?P<y>\d{2})[.-/\s](?P<m>\d{1})[.-/\s](?P<d>\d{2})\s?\)?')
p_day6 = re.compile(r'\s?\(?\s?(20|19)?(?P<y>\d{2})[.-/\s](?P<m>\d{2})[.-/\s](?P<d>\d{2})\s?\)?')
#중간기호는 없고 괄호로 감싸진 경우
p_day_4 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1})(?P<d>\d{1})(?!\d)\s?\)+')#괄호로 감싼거
p_day_5 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1})(?P<d>\d{2})\s?\)+')#괄호로 감싼거
p_day_6 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1,2})(?P<d>\d{1,2})\s?\)+')#괄호로 감싼거

# 사건번호
p_event1=re.compile(r"(?<=\D)(\d{4})\s?([ㄱ-ㅎ가-힣]{1,3})\s?([0-9]+)")
p_event2=re.compile(r"(?<=\D)(\d{2})\s?([ㄱ-ㅎ가-힣]{1,3})\s?([0-9]+)")

docu_list=["", "원인서류", "양도통지서", "집행권원", "강제집행", "등본",
"초본", "외국인증명", "개인회생", "신용회복", "파산", "재산조사", "기타"]
p1=re.compile(r"원인\s?서류|입회\s?신청서")
p2=re.compile(r"양도\s?통지서|채권\s?양도\s?통지서|(?<=[가-힣]{2})양통|\s?양통\s|\s양통\s?")  # ? 는 {0,1}, 세양통신 해결pass
p3=re.compile(r"집행\s?권원|승계\s?(집행문)?|판결문|지급\s?명령|이행\s?권고|화해\s?권고") # count=1이 의미있게 하기 위해 올바른 표현도 넣는다.
p4=re.compile(r"강제\s?집행|(채권)?\s?(추심|압류)?\s?결정(문)?|결정문|(?<!\d)\s?타채|(?<!개시)\s?결정") # 결정이라는 말이 여러곳에서 나올 수 있다.ex개인회생 회생결정

p5=re.compile(r'\(?\s?법인\s?등기부\s?(등본)?\s?\)?|\(?\s?(법인)?\s?등본\s?\)?|\(?\s?주민\s?등록\s?등본\s?\)?')
p5except = re.compile(r"\(?\s?등본.{0,3}원?초본\s?\)?|\(?\s?원?초본.{0,3}등본\s?\)?|\(?\s?등\s?초본\s?\)?|\(?\s?주민\s?등록\s?등본\s?및?\s?초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?및?\s?등본\s?\)?") # '등본 및 초본' 때문에 .{0,3}
p6=re.compile(r"(?<![가-힣])원초본|(?<=원)원초본|(?<![가-힣])\(?\s?원\s?초본\s?\)?|(?<=[가-힣]{3})원초본|\(?\s?초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?\)?") # 이렇게까지 해야되냐? ㅠㅠ
# p7 외국인증명. 컴파일 구문에서 외국인은 젤 끝에 와야 하는 거 유의
p7 = re.compile(r"\(?\s?외국인\s?증명서?\s?\)?|\(?\s?외국인\s?등록\s?사실\s?증명서?\s?\)?|\(?\s?외국인\s?등록증?\s?\)?|\(?\s?외국인\s?\)?")

# 연도 다음에 나오는 개회가 아닌 경우, 전방탐색을 통해 '개인회생'의 '회생'이 걸리는 거 방지. count도 하자
p8=re.compile(r"(?<!\d)개회|개인\s?회생|(?<!개인)회생")
p9=re.compile(r"(?<=[가-힣]{3})신복|[\s_]신복|신용\s?회복") #이름에있는 신복,숫자뒤 신복은 제외. 이름다음에 띄어쓰기 없이 나온 신복은.. 
# p10 파산 pass
p11=re.compile(r"재산\s?조사|재산\s?조회")
p_sign = re.compile("[^a-zA-Zㄱ-ㅎ가-힣0-9\s_()]")
p_etc = re.compile("행자부|재산\s?명시|접수증|집행문\s?부여의\s?소|자동차\s등록\s원부")

# -----------------------------------------------------------

for f in tqdm(files):
    
    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    depth1, depth2, depth3,  = "", "", ""
    key, name, docu, event, extra = "", "", "", "", ""

    # key 없는 거 파일명에러 폴더로
    key = p_key.match(n).group(1)##############################  key

    # 연속 공백 및 기호 제외하기
    n = re.sub("[\s]{2,}", " ", n)

    # 날짜, 주민번호 내부 공백 제거 및 예외 log추가-------
    if p_day4.search(n):
        n = p_day4.sub(r" \g<y>0\g<m>0\g<d>", n)
    elif p_day5.search(n) :
        n = p_day5.sub(r" \g<y>0\g<m>\g<d>", n)
    elif p_day6.search(n) :
        n = p_day6.sub(r" \g<y>\g<m>\g<d>", n)
    elif p_day_4.search(n) :
        n = p_day_4.sub(r" \g<y>0\g<m>0\g<d>", n)
    elif p_day_5.search(n) :
        n = p_day_5.sub(r" \g<y>0\g<m>\g<d>", n)
    elif p_day_6.search(n) :
        n = p_day_6.sub(r" \g<y>\g<m>\g<d>", n)
    else : pass

    # 기호정리
    n = p_sign.sub("", n)

    
    # 사건번호 : 내부 공백 제거 및 연도에 20 없으면 추가
    if p_event1.search(n):
        n=p_event1.sub(r'\1\2\3', n)
        # 1) + 사건번호 연도가 yy라면
    elif p_event2.search(n):
        n=p_event2.sub(r'20\1\2\3', n)
    else : pass 

    e_s = None
    if p_event.search(n) :
        e_s = p_event.search(n).start()
    


    # 문서구분 수정하기 -------------------------------
    # 원인서류
    if re.search("기타", n) : pass
    elif p1.search(n):
        n=p1.sub(docu_list[1], n, count=1)
    # 양도통지서
    elif p2.search(n):
        n=p2.sub(docu_list[2], n, count=1)  # 이름에 양통들어가는 경우가 있을 수 있어서 공백씀..
    # 등초본
    elif p5except.search(n) :
        n=p5except.sub("등초본", n, count=1)
    # 등본(법인등기부등분)
    elif p5.search(n) :
        n=p5.sub(docu_list[5], n, count=1)   
    # 초본
    elif p6.search(n):
        n=p6.sub(docu_list[6], n, count=1)
    # 외국인 증명
    elif p7.search(n):
        n=p7.sub(docu_list[7], n, count=1)
    # 집행권원
    elif p3.search(n[:e_s]):
        n=p3.sub(docu_list[3], n, count=1)
    # 강제집행
    elif p4.search(n[:e_s]):
        n=p4.sub(docu_list[4], n, count=1)
    #개인회생
    elif p8.search(n[:e_s]):
        n=p8.sub(docu_list[8], n, count=1) # 개회 반복될 수 있음
    #신용회복
    elif p9.search(n[:e_s]):
        n=p9.sub(docu_list[9], n, count=1)
    #재산조사
    elif p11.search(n):
        n=p11.sub(docu_list[11], n, count=1)

    #기타 추가하기
    if (p_docu.search(n)==None) and p_etc.search(n) :
        etc = p_etc.search(n)
        rep = "기타 " + etc.group()
        n = p_etc.sub(rep, n)
        

    

# 새 파일명(new_f) 만들기__________________________________________________________
    if p_docu.search(n):
        d_obj = p_docu.search(n)##############################  docu, name
        name = n[8:d_obj.start()].replace("_", " ").strip() #replace가 먼저나와야
        docu = d_obj.group().strip() #양통, 집행권원 때문에 strip 필요
    else :
        print(f, ">>>", n, "다큐없음")

    
    
    if not p_event.search(n) : #사건번호 없으면
        ##############################  extra
        extra=n[d_obj.end():].strip().replace(" ", "_") #strip이 먼저 나와야
    else :  # 사건번호 있으면
        e_obj = p_event.search(n)
        event = e_obj.group()##############################  event
        extra = n[d_obj.end():e_obj.start()].strip().replace(" ", "_") #strip이 먼저 나와야
        extra = extra + "_" +n[e_obj.end():].strip().replace(" ", "_")##############################  extra
        if d_obj.end() > e_obj.start() : # 사건번호앞에 문서구분 없는 경우
            print(f,">>>", n+ext, "문서사 사건 뒤에")
            continue
        else : pass #문서구분 - 사건번호 

    # new_f
    # continue 모두 해줬기 때문에 try 안에만 있으면 된다(현재는 key있는 경우)
    name_items = [key, name, docu]
    if event : name_items.append(event)
    if extra : name_items.append(extra) #일련번호나 _로 끝나는 경우는 re_name에서 제거해줌

    new_f = "_".join(name_items)+ext
    # 마지막에 _ 두개 인 경우 꼭 해줘야 해.
    new_f = re.sub("[_]{2,}", "_", new_f)

    print(f, ">>>", new_f)
    print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}")

##################################################################################################
    # 이름
    depth1 = ""
    depth3=key
    
    # depth1
    for key, value in docu_folder_dict.items():
        if re.search(key, docu):
            depth1=value
            print(f"depth1 :{depth1}")
            break

   

  0%|          | 0/4 [00:00<?, ?it/s]

***접수증_***
--
20450092_전순희_접수증_2022본553.pdf >>> 20450092_전순희_기타_2022본553_접수증_.pdf
key:20450092, name:전순희, docu:기타, event:2022본553, extra:접수증__
depth1 :기타
***접수증_***
-에혀-
20450092_전순희_접수증_2022본553에혀.pdf >>> 20450092_전순희_기타_2022본553_접수증_에혀.pdf
key:20450092, name:전순희, docu:기타, event:2022본553, extra:접수증__에혀
depth1 :기타
***접수증***
-에혀-
20450092 전순희 접수증 2022본553 에혀.pdf >>> 20450092_전순희_기타_2022본553_접수증_에혀.pdf
key:20450092, name:전순희, docu:기타, event:2022본553, extra:접수증_에혀
depth1 :기타
20438985_정병수_기타행자부.pdf >>> 20438985_정병수_기타_행자부.pdf
key:20438985, name:정병수, docu:기타, event:, extra:행자부
depth1 :기타


In [138]:
files = ["dfabefc", "cd", "efc"]
p6=re.compile(r"(?<![가-힣])원초본|(?<=원)원초본|(?<![가-힣])\(?\s?원\s?초본\s?\)?|(?<=[가-힣]{3})원초본|\(?\s?초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?\)?") # 이렇게까지 해야되냐? ㅠㅠ
p6.search("전석원초본")


<re.Match object; span=(3, 5), match='초본'>